In [2]:
%matplotlib inline
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, linewidth=75)

In [3]:
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [4]:
def model(t_u, w, b):
    return w * t_u + b

In [5]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c) ** 2
    return squared_diffs.mean()

In [6]:
w = torch.ones(())
b = torch.zeros(())

t_p = model(t_u, w, b)
t_p

tensor([35.7000, 55.9000, 58.2000, 81.9000, 56.3000, 48.9000, 33.9000,
        21.8000, 48.4000, 60.4000, 68.4000])

In [7]:
loss = loss_fn(t_p, t_c)
loss

tensor(1763.8846)

In [8]:
x = torch.ones(())
y = torch.ones(3, 1)
z = torch.ones(1, 3)
a = torch.ones(2, 1, 1)
print(f"shapes: x: {x.shape}, y: {y.shape}")
print(f"        z: {z.shape}, a: {a.shape}")
print("x * y:", (x * y).shape)
print("y * z:", (y * z).shape)
print("y * z * a:", (y * z * a).shape)

shapes: x: torch.Size([]), y: torch.Size([3, 1])
        z: torch.Size([1, 3]), a: torch.Size([2, 1, 1])
x * y: torch.Size([3, 1])
y * z: torch.Size([3, 3])
y * z * a: torch.Size([2, 3, 3])


In [9]:
delta = 0.1

# 重み w に関する損失関数の勾配を有限差分法（中心差分）で数値的に近似
loss_rate_of_change_w = (
    loss_fn(model(t_u, w + delta, b), t_c) - loss_fn(model(t_u, w - delta, b), t_c)
) / (2.0 * delta)

In [10]:
learning_rate = 1e-2

# 数値微分で得られた勾配に基づき、重み w を勾配降下法で更新
w = w - learning_rate * loss_rate_of_change_w

In [11]:
# バイアス b に関する損失の勾配を有限差分で近似
loss_rate_of_change_b = (
    loss_fn(model(t_u, w, b + delta), t_c) - loss_fn(model(t_u, w, b - delta), t_c)
) / (2.0 * delta)

# 勾配に基づいてバイアス b を勾配降下法で更新
b = b - learning_rate * loss_rate_of_change_b

In [12]:
def dloss_fn(t_p, t_c):
    # MSE損失に対する予測値 t_p の導関数（∂L/∂t_p）を計算
    dsq_diffs = 2 * (t_p - t_c) / t_p.size(0)
    return dsq_diffs

In [13]:
def dmodel_dw(t_u, w, b):
    # 線形モデル y = w * t_u + b における w に関する偏微分（∂y/∂w）を返す
    return t_u

In [14]:
def dmodel_db(t_u, w, b):
    # 線形モデル y = w * t_u + b における b に関する偏微分（∂y/∂b）は常に 1
    return 1.0

In [15]:
def grad_fn(t_u, t_c, t_p, w, b):
    # 損失関数（MSE）の予測値に関する導関数を取得（∂L/∂t_p）
    dloss_dtp = dloss_fn(t_p, t_c)

    # 連鎖律により ∂L/∂w = ∂L/∂t_p × ∂t_p/∂w
    dloss_dw = dloss_dtp * dmodel_dw(t_u, w, b)

    # 同様に ∂L/∂b = ∂L/∂t_p × ∂t_p/∂b
    dloss_db = dloss_dtp * dmodel_db(t_u, w, b)

    # バッチ全体の勾配を合計して、wとbの勾配ベクトルとして返す
    return torch.stack([dloss_dw.sum(), dloss_db.sum()])